<a href="https://colab.research.google.com/github/mahidedhia/ResumeParser-NLP/blob/master/ResumeParsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing Libraries

In [30]:
!pip install pdfminer.six
!pip install pretty_html_table
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import spacy
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io
import numpy as np
import nltk
import pandas as pd
import dateutil
import re
import json
from pretty_html_table import build_table
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_sm")

#for stopwords required for preprocessing
stopwords = nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##PDF to Text Conversion

In [32]:
filename = "QA-Manual-Tester.pdf"

In [33]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        # iterate over all pages of PDF document
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            # creating a resoure manager
            resource_manager = PDFResourceManager()
            
            # create a file handle
            fake_file_handle = io.StringIO()
            
            # creating a text converter object
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )

            # creating a page interpreter
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )

            # process current page
            page_interpreter.process_page(page)
            
            # extract text
            text = fake_file_handle.getvalue()
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()

In [34]:
resume_text = ''
# calling above function and extracting text
for page in extract_text_from_pdf(filename):
    resume_text += page + ' '

resume_text

'FIRST LAST\nBay Area, California • +1-234-456-789 • professionalemail@resumeworded.com • linkedin.com/in/username\n\nPROFESSIONAL EXPERIENCE\n\nResume Worded, New York, NY\nQA Manual Tester\n\nJun 2018 – Present\n\n● Enabled critical test case complexity metrics with support for Rapid adoption of  functional automation using\n\na scriptless test case adaptor by standardizing a Test Case construction method that was built\nAutomation-ready & supported a test automation framework leading to a 45% increase in reusability with\nreductions in TCO approaching 25%.\n\n● Optimized scripting, modularity, & maintenance which resulted in an 18% decrease in workflow friction.\n● Increased the company’s ability to take and complete projects without increasing manpower by 15% by\n\nreducing QA testing turnaround time by 30%.\n\nGrowthsi, New York, NY\nQA Manual Tester\n\nJan 2015 – May 2018\n\n● Restructured utilities & improved the process documentation leading to a 40% reduction in client support

In [35]:
#removing unwanted unicode characters
resume_text = resume_text.encode("ascii", "ignore")
resume_text = resume_text.decode()
resume_text

'FIRST LAST\nBay Area, California  +1-234-456-789  professionalemail@resumeworded.com  linkedin.com/in/username\n\nPROFESSIONAL EXPERIENCE\n\nResume Worded, New York, NY\nQA Manual Tester\n\nJun 2018  Present\n\n Enabled critical test case complexity metrics with support for Rapid adoption of  functional automation using\n\na scriptless test case adaptor by standardizing a Test Case construction method that was built\nAutomation-ready & supported a test automation framework leading to a 45% increase in reusability with\nreductions in TCO approaching 25%.\n\n Optimized scripting, modularity, & maintenance which resulted in an 18% decrease in workflow friction.\n Increased the companys ability to take and complete projects without increasing manpower by 15% by\n\nreducing QA testing turnaround time by 30%.\n\nGrowthsi, New York, NY\nQA Manual Tester\n\nJan 2015  May 2018\n\n Restructured utilities & improved the process documentation leading to a 40% reduction in client support\n\nticket

##Pre-Processing

In [36]:
#function for tokenizing
def tokenize(text):
  #using spacy
  doc = nlp(text)
  word_tokens = []
  for token in doc:
    word_tokens.append(str(token))
  return word_tokens

In [37]:
#function for removing stop words and special characters. also converting tokens to lowercase
def stop_words(word_tokens):
  #using nltk
  special_char='\"[,@_!$%^&*()<>?/\|}{~:]#.-\''
  removed_stop_words = [word.lower() for word in word_tokens if word not in stops and str(word) not in special_char and not (ord(str(word)[0]) >= 0 and ord(str(word)[0]) <= 32)] 
  return removed_stop_words

In [38]:
def preProcessess(text):
  #removing \n
  text = text.replace('\n', ' ')
  #word tokenzing
  word_tokens=tokenize(text)
  #removal of stopwords and special characters
  removed_stop_words = stop_words(word_tokens)
  return removed_stop_words

In [39]:
preProcessedTokens = preProcessess(resume_text)
preProcessedTokens

['first',
 'last',
 'bay',
 'area',
 'california',
 '+1',
 '234',
 '456',
 '789',
 'professionalemail@resumeworded.com',
 'linkedin.com/in/username',
 'professional',
 'experience',
 'resume',
 'worded',
 'new',
 'york',
 'ny',
 'qa',
 'manual',
 'tester',
 'jun',
 '2018',
 'present',
 'enabled',
 'critical',
 'test',
 'case',
 'complexity',
 'metrics',
 'support',
 'rapid',
 'adoption',
 'functional',
 'automation',
 'using',
 'scriptless',
 'test',
 'case',
 'adaptor',
 'standardizing',
 'test',
 'case',
 'construction',
 'method',
 'built',
 'automation',
 'ready',
 'supported',
 'test',
 'automation',
 'framework',
 'leading',
 '45',
 'increase',
 'reusability',
 'reductions',
 'tco',
 'approaching',
 '25',
 'optimized',
 'scripting',
 'modularity',
 'maintenance',
 'resulted',
 '18',
 'decrease',
 'workflow',
 'friction',
 'increased',
 'companys',
 'ability',
 'take',
 'complete',
 'projects',
 'without',
 'increasing',
 'manpower',
 '15',
 'reducing',
 'qa',
 'testing',
 'turnar

### Custom NER model

In [40]:
df = pd.read_json('job-titles.json')
df=df.rename(columns={'job-titles': 'text'})
df['tag']="job-role"

df_companies=pd.read_csv("companies.csv")

rows=[]
for i in range(len(df_companies)):
  row = {"text": df_companies.loc[i, "name"], "tag":"organization"}
  rows.append(row)
df = df.append(rows, ignore_index=True)

print(df)

                                    text           tag
0                      1st grade teacher      job-role
1                           1st pressman      job-role
2              1st pressman on web press      job-role
3                              21 dealer      job-role
4                      2nd grade teacher      job-role
...                                  ...           ...
215079                              mapn  organization
215080           egb systems & solutions  organization
215081                beztak corporation  organization
215082    inventory locator service, llc  organization
215083  toyota material handling belgium  organization

[215084 rows x 2 columns]


<ipython-input-40-a6f91cae10f0>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(rows, ignore_index=True)


In [41]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split

df = df.sample(frac=1).reset_index(drop=True)  # shuffle the dataset
X = df.text
y = df.tag
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

tag_model = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
tag_model.fit(X_train, y_train)


from sklearn.metrics import classification_report, accuracy_score
y_pred = tag_model.predict(X_test)
accuracy=accuracy_score(y_pred, y_test)
print(accuracy)

0.9710659269131823


##Extraction

In [42]:
def isDate(string, fuzzy=False):
    try: 
        dateutil.parser.parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

isDate("18")
isDate("13th June 2022 - 5th Aug 2022")

False

In [43]:
nlp = spacy.load("en_core_web_sm")

# A complete sentence or a link cannot be a header
def isHeader(text):
  # A complete sentence contains at least one subject, one predicate, one object, and closes with punctuation. 
  # Subject and object are almost always nouns, and the predicate is always a verb.
  text=nlp(text)
  has_noun = 2
  has_verb = 1
  for token in text:
    if token.pos_ in ["NOUN", "PROPN", "PRON"]:
        has_noun -= 1
    elif token.pos_ == "VERB":
        has_verb -= 1
  if has_noun < 1 and has_verb < 1:
    return False
  # check for link
  pattern=r'(?P<url>https?://[^\s]+)'
  links = re.findall(pattern, str(text))
  if links:
    return False
  return True

# splits the string at each match of: '|', '@' or at
def partitionExperienceSubheader(string):
  pattern = r'\||@|\bat\b'
  result = filter(None, re.split(pattern, string))
  return list(result)

test_strings = [
    "Resume Worded | June 2022 - Present",
    "App developer at Resume Worded",
    "App developer @ Resume Worded",
    "App developer @Resume Worded",
    "Link: https://www.geeksforgeeks.org/",
    "Nurturing Lives | Feb 2020",
    "K. J. Somaiya College of Engineering",
]
for string in test_strings:
    print(partitionExperienceSubheader(string))
    print("Is Header: ", isHeader(string), "\n")

['Resume Worded ', ' June 2022 - Present']
Is Header:  True 

['App developer ', ' Resume Worded']
Is Header:  True 

['App developer ', ' Resume Worded']
Is Header:  True 

['App developer ', 'Resume Worded']
Is Header:  True 

['Link: https://www.geeksforgeeks.org/']
Is Header:  False 

['Nurturing Lives ', ' Feb 2020']
Is Header:  False 

['K. J. Somaiya College of Engineering']
Is Header:  True 



In [44]:
headers_experience = (
        'career profile',
        'employment history',
        'work history',
        'work experience',
        'experience',
        'professional experience',
        'professional background',
        'additional experience',
        'career related experience',
        'related experience',
        'programming experience',
        'freelance',
        'freelance experience',
        'army experience',
        'military experience',
        'military background',
)
headers_volunteering = (
        "volunteer",
        "volunteering",
)
headers_education = (
        'academic background',
        'academic experience',
        'programs',
        'courses',
        'related courses',
        'education',
        'qualifications',
        'educational background',
        'educational qualifications',
        'educational training',
        'education and training',
        'training',
        'academic training',
        'professional training',
        'course project experience',
        'related course projects',
        'internship experience',
        'internships',
        'apprenticeships',
        'college activities',
        'certifications',
        'special training',
    )
headers_skills = (
        'credentials',
        'areas of experience',
        'areas of expertise',
        'areas of knowledge',
        'skills',
        "other skills",
        "other abilities",
        'career related skills',
        'professional skills',
        'specialized skills',
        'technical skills',
        'computer skills',
        'personal skills',
        'interpersonal skills'
        'knowledge',        
        'technologies',
        'technical experience',
        'proficiencies',
        'languages',
        'language competencies and skills',
        'programming languages',
        'competencies'
    )
headers_projects = (
    'projects',
    'personal projects',
    'academic projects',
    'freelance projects',
)
headers_achievements = (
    "other achievements",
    "other accomplishments",
    "previous achievements",
    "previous accomplishments",
    "more achievements",
    "more accomplishments"
)
headers_contact = (
    "contact",
    "Reach me",
    "connect",
)

Cosine Similarity

In [45]:
def getSimilarity(all_values, user_values, threshold, flag):
  vectorizer = TfidfVectorizer()
  vectorizer.fit(all_values)
  filtered_values = []
  score = 0

  for skill in user_values:
    test_word_vector = vectorizer.transform([skill])
    similarities = cosine_similarity(test_word_vector, vectorizer.transform(all_values))
    if similarities.max() > threshold:
            if flag==1:
              filtered_values.append(skill)
            else:
              score = score + similarities.max()

  if flag==1:
    return filtered_values
  else:
    return score

Phrase Matcher

In [46]:
def getMatcher(file,col,file_flag):
  if file_flag==1:
    df = pd.read_csv(file)
  else:
    df = pd.read_excel(file)

  column = df[col]
  df = pd.DataFrame(column)
  all_values = df[col].astype(str).values.tolist()
  all_values_lower = [x.lower() for x in all_values]
  nlp = spacy.load("en_core_web_sm")
  phrases = all_values_lower
  matcher = PhraseMatcher(nlp.vocab)
  patterns = [nlp.make_doc(phrase) for phrase in phrases]
  matcher.add("PHRASES", None, *patterns)
  return matcher

def getMatchedPhrases(doc,matcher):
  matches = matcher(doc)
  values=[]
  for match_id, start, end in matches:         
      matched_phrase = doc[start:end]
      values.append(str(matched_phrase))
  return values


In [47]:
class ResumeParser:

  def __init__(self, text):
    self.resume_text = text
    self.lines = text.split("\n")
  
  resume_sections = {
      'name': {},
      'experience': {},
      'volunteering': {},
      'education': {},
      'skills': {},
      'projects': {},
      'achievements': {},
      'contact': {}
  }
  header_indices = {}

  #Obtain line nos of Headers 
  def getHeaderIndices(self):
    for i, line in enumerate(self.lines):
      if len(line):
        #Header always starts in Uppercase
        if line[0].isupper():
          #The line is a header if it matches the required section headers and isn't a complete sentence
          if [item for item in headers_experience if line.lower().startswith(item)] and isHeader(line):
            self.header_indices[i]="experience"
          elif [item for item in headers_volunteering if line.lower().startswith(item)] and isHeader(line):
            self.header_indices[i]="volunteering"
          elif [item for item in headers_education if line.lower().startswith(item)] and isHeader(line):
            self.header_indices[i]="education"
          elif [item for item in headers_skills if line.lower().startswith(item)] and isHeader(line):
            self.header_indices[i]="skills"
          elif [item for item in headers_projects if line.lower().startswith(item)] and isHeader(line):
            self.header_indices[i]="projects"
          elif [item for item in headers_achievements  if line.lower().startswith(item)] and isHeader(line):
            self.header_indices[i]="achievements"
          elif [item for item in headers_contact if line.lower().startswith(item)] and isHeader(line):
            self.header_indices[i]="contact"

  #Obtain raw text for different resume sections using header indices/linenos
  def getRawResumeSections(self):
    no_lines = len(self.lines)
    header_linenos = list(self.header_indices.keys())
    list_last_index = len(header_linenos)-1
    for counter, lineno in enumerate(header_linenos):
      start_index = lineno+1
      if (counter<list_last_index):
        end_index = header_linenos[counter+1]
      else:
        end_index = no_lines
      self.resume_sections[self.header_indices[lineno]]=self.lines[start_index:end_index]

  def getExperience(self, section_string):
    if not bool(self.resume_sections[section_string]):
      return
    text = self.resume_sections[section_string]
    self.resume_sections[section_string] = [] #list of dictionaries for details of experiences
    exp={"description":""}
    go_to_next=False
    for i, line in enumerate(text):
      if len(line):
        if line[0].isupper():
          # if line isn't a complete sentence, it can be either org, job role or date
          if isHeader(line):
            line_parts = partitionExperienceSubheader(line)
            for line_part in line_parts:
              if isDate(str(nlp(line_part)[0:2])):
                exp['duration']=line_part
              else:
                if go_to_next:
                  self.resume_sections[section_string].append(exp) #add to list before moving to next
                  exp={"description":""}
                  go_to_next=False
                # if not a date, apply model to predict tag
                exp[tag_model.predict([line_part])[0]]=line_part
          else:
            exp["description"]+=line
            go_to_next=True #Exp description is written at last, after which next experience appears
        else:
            exp["description"]+=line
            go_to_next=True
    self.resume_sections[section_string].append(exp)

  
  def getEducation(self):
      parsed_resume_edu=[]
  
      matcher_b = getMatcher('Major_or_Branch.csv','Major',1)
      matcher_d = getMatcher('Academic Degrees.xlsx','Degree',2)

      # Test the matcher
      for edu in self.resume_sections["education"]:
        
        parsed_branch_degree={}
        edu = edu.lower()
        edu_new = edu.replace(".", "")

        doc = nlp(edu_new)
        
        degrees = getMatchedPhrases(doc,matcher_d)
        if len(degrees)!=0:
          parsed_branch_degree["degree"] = degrees

        branches = getMatchedPhrases(doc,matcher_b)
        if len(branches)!=0:
          parsed_branch_degree["branch"] = branches
       

        if len(parsed_branch_degree)!=0:
          parsed_resume_edu.append(parsed_branch_degree)    
      return parsed_resume_edu      

  #get skills from resume
  def getSkills(self):
        file ='Technology Skills.xlsx'
        df = pd.read_excel(file)
        column = df['Example']
        df = pd.DataFrame(column)
        all_skills = df['Example'].astype(str).values.tolist()
        all_skills_lower = [x.lower() for x in all_skills]

        resume_skills =""

        for skill in self.resume_sections["skills"]:
          resume_skills =  resume_skills + " "+skill

        resume_skills =  resume_skills.encode("ascii", "ignore")
        resume_skills =  resume_skills.decode()
        resume_skills_tokens = preProcessess(resume_skills)
      
        return getSimilarity(all_skills_lower, resume_skills_tokens, 0.6, 1) 

  #method to parse the entire resume - calls all the above methods of the class
  def getResumeData(self):
    self.getHeaderIndices()
    if len(self.header_indices)!=0:
      self.getRawResumeSections()
      self.getExperience("experience")
      self.getExperience("volunteering")
      self.parsed_resume_education=self.getEducation()
      self.parsed_resume_skills = self.getSkills()
    return self.resume_sections

In [48]:
obj = ResumeParser(resume_text)
obj.getResumeData()
print()

In [49]:
print(json.dumps(obj.resume_sections["experience"], indent=4, sort_keys=True))
print(json.dumps(obj.resume_sections["volunteering"], indent=4, sort_keys=True))
print(json.dumps(obj.parsed_resume_education, indent=4, sort_keys=True))
print(json.dumps(obj.parsed_resume_skills, indent=4, sort_keys=True))

[
    {
        "description": " Enabled critical test case complexity metrics with support for Rapid adoption of  functional automation usinga scriptless test case adaptor by standardizing a Test Case construction method that was builtAutomation-ready & supported a test automation framework leading to a 45% increase in reusability withreductions in TCO approaching 25%. Optimized scripting, modularity, & maintenance which resulted in an 18% decrease in workflow friction. Increased the companys ability to take and complete projects without increasing manpower by 15% byreducing QA testing turnaround time by 30%.",
        "duration": "Jun 2018  Present",
        "job-role": "QA Manual Tester",
        "organization": "Resume Worded, New York, NY"
    },
    {
        "description": " Restructured utilities & improved the process documentation leading to a 40% reduction in client supporttickets & an 80% increase in uptime. Achieved department-wide improvement metrics based on QA scorecard

Comparison of Resume skills with JD skills

In [50]:
#JD with each skill on new line in text file

with open('JD_Skills.txt', 'r') as file:
    jd_skills_tokens = [line.strip() for line in file]

jd_skills_tokens = [i for i in jd_skills_tokens if i] #remove empty strings
jd_skills_tokens = [x.lower() for x in jd_skills_tokens]

resume_score =  getSimilarity(jd_skills_tokens, obj.parsed_resume_skills,0.5,2)

matching_per = (resume_score/len(jd_skills_tokens))*100

print("Skills Matching % with JD: "+str(matching_per))

Skills Matching % with JD: 31.547005383792516


In [51]:
# converting output to HTML
with open('output.html', 'a') as f:
    print_list = []
    print_list.append(pd.DataFrame(obj.resume_sections["experience"], columns=['organization', 'job-role', 'duration', 'description']))
    print_list.append(pd.DataFrame(obj.resume_sections["volunteering"], columns=['organization', 'job-role', 'duration', 'description']))
    for item in print_list:
      if not item.empty:
        html_table_blue_light = build_table(item, 'blue_light')
        f.write(html_table_blue_light)